# Assignment 1: Data Parsing, Cleansing and Integration
## Task 3

Environment: Python 3 and Jupyter notebook

Main Libraries used in Task 3:
* pandas
* re
* numpy
* difflib
* datetime


## Introduction
#### Goal of Task 3:
The goal of Task 3 is to integrate the job posting data from a separate source with the cleaned data from the output of Task 1 & 2. The expected end result is a clean joined dataset which is free of duplicates, based on a global key.

#### Problems encountered:
The problems encountered included:
* Different field names and field values, requiring normalisation to align similar fields e.g. Category fields
* NULL values in the provided dataset, requiring minor cleaning
* Company names across the two datasets having slight variances, requiring cross-checks across the Task2 and Task 3 fields and similarity score analysis
* Duplicates after merging

#### Approaches to handle data errors:
* Univariate analysis to understand the distribution of fields
* Renaming of fields and conversion of numerical categorical values to text values
* Checking for duplicates and NULL values
* Similarity score analysis to find similar company names
* Creation of a global data key

##  Import libraries 

In [1]:
# Code to import libraries as you need in this assessment

# Data cleaning and tidying libraries
import numpy as np #linear algebra
import pandas as pd #datapreprocessing, CSV file I/O
import difflib # comparing text
from datetime import datetime

# Regular Expression Library
import re

# Display DataFrame without scientific notation
pd.set_option('display.float_format', '{:.6f}'.format)

# Display all columns and rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

## 1. Examining and loading data

In Task 1, we examine the provided "\<student\_id\>_dataset2.csv" the structure and schema. This includes some simple descriptive analytics and adjustments.

The steps in this task include:
* 1.1 Re-arrange the columns of task 3 dataset so they are in the same order as Task 2 Solution

In [2]:
# Read in task 3 dataset in csv format
df_task3 = pd.read_csv('s3932835_dataset2.csv')

# read in output from task 1 and 2 in csv format
df_task2 = pd.read_csv('s3932835_dataset1_solution.csv')

In [3]:
df_task2.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,67770713,CNC Programmer (Offline),birmingham,non-specified,non-specified,non-specified,Engineering Jobs,32000.000000,2013-06-10 15:00:00,2013-07-10 15:00:00,britishjobsonthe.net
1,69026322,Weekend Engineering Team Leader,essex,non-specified,non-specified,non-specified,Engineering Jobs,39000.000000,2012-05-15 12:00:00,2012-05-29 12:00:00,britishjobsonthe.net
2,67770033,Junior Writer,nottingham,non-specified,non-specified,non-specified,"PR, Advertising & Marketing Jobs",16500.000000,2013-06-17 00:00:00,2013-07-17 00:00:00,britishjobsonthe.net
3,69025763,Net Developer Manchester (Up To pound;****K ...,manchester,non-specified,non-specified,non-specified,IT Jobs,38000.000000,2012-02-21 12:00:00,2012-03-06 12:00:00,britishjobsonthe.net
4,68017718,Chef De Partie pound;****gratutities****AA Ros...,bath,non-specified,non-specified,non-specified,Hospitality & Catering Jobs,16000.000000,2013-03-27 15:00:00,2013-04-10 15:00:00,britishjobsonthe.net


In [4]:
df_task3.head()

,Monthly Payment,Closing,Organisation,Location,Category,Full-Time Equivalent (FTE),Opening,Job Title,Type
0,1200.000000,2012-12-27 12:00:00,Protocol Education,UK,Education,0.800000,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN
1,3125.000000,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.000000,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent
2,4333.330000,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.000000,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent
3,3750.000000,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.000000,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN
4,2916.670000,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.000000,2012-12-29 12:00:00,Channel Account Manager Software,NaN


In [5]:
df_task3.describe()

,Monthly Payment,Full-Time Equivalent (FTE)
count,5000.000000,5000.000000
mean,2888.458606,0.983360
std,1347.913223,0.098372
min,417.170000,0.200000
25%,1920.000000,1.000000
50%,2605.000000,1.000000
75%,3541.670000,1.000000
max,8000.000000,1.000000


### 1.1 Re-arrange the columns of task 3 dataset so they are in the same order as Task 2 solution

In [6]:
# get column names of df_task3
df_task3.columns


Index(['Monthly Payment', 'Closing', 'Organisation', 'Location', 'Category',
       'Full-Time Equivalent (FTE)', 'Opening', 'Job Title', 'Type'],
      dtype='object')

In [7]:
# get column names of df_task3
df_task2.columns

Index(['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime',
       'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName'],
      dtype='object')

In [8]:
# rename columns in df_task3 to match df_task2

# Use the rename() method to rename columns
#df = df.rename(columns={'A': 'NewA', 'B': 'NewB'})

df_task3['Id'] = range(1, len(df_task3) + 1)
df_task3['SourceName'] = 'www.jobhuntlisting.com'

df_task3.rename(columns={'Id':'Id'
                         , 'Job Title':'Title'
                         , 'Location':'Location'
                         , 'Organisation':'Company'
                         , 'Full-Time Equivalent (FTE)':'ContractType'
                         , 'Type':'ContractTime'
                         ,'Category':'Category'
                         , 'Monthly Payment':'Salary'
                         , 'Opening':'OpenDate'
                         , 'Closing':'CloseDate'
                         , 'SourceName':'SourceName'
                         }, inplace=True)
df_task3 = df_task3[['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime',
       'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName']]


In [9]:
df_task3.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,1,Higher Level Teaching Assistant (HLTA),UK,Protocol Education,0.800000,NaN,Education,1200.000000,2012-11-27 12:00:00,2012-12-27 12:00:00,www.jobhuntlisting.com
1,2,Software Developers / Software Architects Not...,UK,Cordius Ltd,1.000000,Permanent,Information Technology,3125.000000,2013-06-26 15:00:00,2013-08-25 15:00:00,www.jobhuntlisting.com
2,3,Senior BI Consultant (Cognos),Walton-On-Thames,Understanding Recruitment,1.000000,Permanent,Information Technology,4333.330000,2012-10-18 15:00:00,2013-01-16 15:00:00,www.jobhuntlisting.com
3,4,"Technical Sales Manager, Pigments Fillers to S...",UK,Wallace Hind Selection,1.000000,NaN,Sales,3750.000000,2012-12-15 15:00:00,2013-02-13 15:00:00,www.jobhuntlisting.com
4,5,Channel Account Manager Software,Eastern England,Greenhill Group Ltd,1.000000,NaN,Sales,2916.670000,2012-12-29 12:00:00,2013-01-12 12:00:00,www.jobhuntlisting.com


## 2. Resolving conflicts

In Tast 2, we are analysing the 'conflicts' between the two datasets and aligning the fields and data values to have similar categorical names. There is some similarity score analysis to convert the company name to remove small spelling differences and align the two datasets. Both datasets are also scanned for missing values.

In this task, the below steps include:
* 2.1 Converting ContractType and Contract Time Values
* 2.2 Convert salary from monthly to annual value
* 2.3 Convert company name
* 2.4 Checking other columns, one last time.

### 2.1 Converting Contract Type and Contract Time Values

In [10]:
df_task3.ContractType.value_counts()

1.000000    4834
0.400000      47
0.800000      41
0.600000      39
0.200000      39
Name: ContractType, dtype: int64

In [11]:
# Change ContractType to Full Time if ==1 or  Part Time if <1 else 'non-specified'
df_task3['ContractType'] = np.where(df_task3['ContractType'] == 1, 'full_time', 'part_time')
df_task3.ContractType.value_counts()

full_time    4834
part_time     166
Name: ContractType, dtype: int64

In [12]:
df_task3.ContractTime.value_counts()

Permanent              3040
Fixed Term Contract     540
Name: ContractTime, dtype: int64

In [13]:
# Define a function to apply the if condition
def update_contract_time(row):
    if row['ContractTime'] == 'Permanent':
        return 'permanent'
    elif row['ContractTime'] == 'Fixed Term Contract':
        return 'contract'
    else:
        return 'non-specified'

# Apply the function to each row using apply()
df_task3['ContractTime'] = df_task3.apply(update_contract_time, axis=1)
df_task3.ContractTime.value_counts()


permanent        3040
non-specified    1420
contract          540
Name: ContractTime, dtype: int64

In [14]:
# look at category in df_task3 v df_task2
print(df_task3.Category.unique())
print("\n")
print(df_task2.Category.unique())

['Education' 'Information Technology' 'Sales' 'Finance' 'Engineering'
 'Health' 'Hospitality' 'Marketing']


['Engineering Jobs' 'PR, Advertising & Marketing Jobs' 'IT Jobs'
 'Hospitality & Catering Jobs' 'Teaching Jobs' 'Accounting & Finance Jobs'
 'Sales Jobs' 'Healthcare & Nursing Jobs']


In [15]:
# create a dictionary that maps the category in df_task3 to df_task2
category_dict = {'Education':'Teaching Jobs'
                 , 'Information Technology': 'IT Jobs'
                 , 'Sales': 'Sales Jobs' 
                 , 'Finance':'Accounting & Finance Jobs'
                 , 'Engineering':'Engineering Jobs'
                 , 'Health' 'Hospitality':'Hospitality & Catering Jobs'
                 , 'Marketing':'PR, Advertising & Marketing Jobs'
                }

# map this new loopup to df_task3
df_task3['Category'] = df_task3['Category'].map(category_dict)
df_task3.Category.value_counts()

IT Jobs                             1317
Engineering Jobs                     744
Accounting & Finance Jobs            668
Sales Jobs                           498
Teaching Jobs                        326
PR, Advertising & Marketing Jobs     259
Name: Category, dtype: int64

### 2.2 Convert Salary from Monthly to Annual Value

In [16]:
# Convert salary into annual salary
df_task3['Salary'].describe()

count   5000.000000
mean    2888.458606
std     1347.913223
min      417.170000
25%     1920.000000
50%     2605.000000
75%     3541.670000
max     8000.000000
Name: Salary, dtype: float64

In [17]:
df_task3['Salary'] = df_task3['Salary']*12
df_task3['Salary'].describe()

count    5000.000000
mean    34661.503272
std     16174.958673
min      5006.040000
25%     23040.000000
50%     31260.000000
75%     42500.040000
max     96000.000000
Name: Salary, dtype: float64

In [18]:
df_task3.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,1,Higher Level Teaching Assistant (HLTA),UK,Protocol Education,part_time,non-specified,Teaching Jobs,14400.000000,2012-11-27 12:00:00,2012-12-27 12:00:00,www.jobhuntlisting.com
1,2,Software Developers / Software Architects Not...,UK,Cordius Ltd,full_time,permanent,IT Jobs,37500.000000,2013-06-26 15:00:00,2013-08-25 15:00:00,www.jobhuntlisting.com
2,3,Senior BI Consultant (Cognos),Walton-On-Thames,Understanding Recruitment,full_time,permanent,IT Jobs,51999.960000,2012-10-18 15:00:00,2013-01-16 15:00:00,www.jobhuntlisting.com
3,4,"Technical Sales Manager, Pigments Fillers to S...",UK,Wallace Hind Selection,full_time,non-specified,Sales Jobs,45000.000000,2012-12-15 15:00:00,2013-02-13 15:00:00,www.jobhuntlisting.com
4,5,Channel Account Manager Software,Eastern England,Greenhill Group Ltd,full_time,non-specified,Sales Jobs,35000.040000,2012-12-29 12:00:00,2013-01-12 12:00:00,www.jobhuntlisting.com


In [19]:
# round salary in both tables to whole numbers
df_task3['Salary'] = df_task3['Salary'].round(0)
df_task3['Salary'].describe()


count    5000.000000
mean    34661.503400
std     16174.958437
min      5006.000000
25%     23040.000000
50%     31260.000000
75%     42500.000000
max     96000.000000
Name: Salary, dtype: float64

In [20]:

df_task2['Salary'] = df_task2['Salary'].round(0)
df_task2['Salary'].describe()

count    50753.000000
mean     34449.278683
std      15516.472379
min       5000.000000
25%      23400.000000
50%      31500.000000
75%      42500.000000
max     108725.000000
Name: Salary, dtype: float64

### 2.4 Convert Company name

In [21]:
# convert to lower case
df_task3['Company'] = df_task3['Company'].str.lower()

# strip the white spaces from the beginning and end of the company name
df_task3['Company'] = df_task3['Company'].str.strip()

In [22]:
# Define a function to clean the text from punctuation
def clean_text(text):
    if isinstance(text, str):  # Check if the input is a string
        return re.sub(r'[^\w\s]', '', text)
    else:
        return ''

# Apply the clean_text function to the 'Company' column
df_task3['Company'] = df_task3['Company'].apply(clean_text)

In [23]:
# Remove the company endings or suffices from the company name
# Create a regular expression pattern for company endings at the end of the string
company_endings = ['ltd', 'limited', 'plc', 'inc', 'incorporated', 'corp', 'corporation', 'llc', 'llp', 'lp', 'pvt', 'private', 'pty', 'pty ltd', 'gmbh', 'sarl', 'sa', 'sas', 'trust', 'group']
pattern = r'\s*(?:' + '|'.join(company_endings) + r')$'

# Define a function to clean the text
def clean_endings(text):
    if isinstance(text, str):  # Check if the input is a string
        return re.sub(pattern, '', text, flags=re.IGNORECASE).strip()
    else:
        return ''


# Apply the clean_endings function to the 'Company' column
df_task3['Company'] = df_task3['Company'].apply(clean_endings)
    

In [24]:
df_task3.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,1,Higher Level Teaching Assistant (HLTA),UK,protocol education,part_time,non-specified,Teaching Jobs,14400.000000,2012-11-27 12:00:00,2012-12-27 12:00:00,www.jobhuntlisting.com
1,2,Software Developers / Software Architects Not...,UK,cordius,full_time,permanent,IT Jobs,37500.000000,2013-06-26 15:00:00,2013-08-25 15:00:00,www.jobhuntlisting.com
2,3,Senior BI Consultant (Cognos),Walton-On-Thames,understanding recruitment,full_time,permanent,IT Jobs,52000.000000,2012-10-18 15:00:00,2013-01-16 15:00:00,www.jobhuntlisting.com
3,4,"Technical Sales Manager, Pigments Fillers to S...",UK,wallace hind selection,full_time,non-specified,Sales Jobs,45000.000000,2012-12-15 15:00:00,2013-02-13 15:00:00,www.jobhuntlisting.com
4,5,Channel Account Manager Software,Eastern England,greenhill group,full_time,non-specified,Sales Jobs,35000.000000,2012-12-29 12:00:00,2013-01-12 12:00:00,www.jobhuntlisting.com


In [25]:
df_task2.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,67770713,CNC Programmer (Offline),birmingham,non-specified,non-specified,non-specified,Engineering Jobs,32000.000000,2013-06-10 15:00:00,2013-07-10 15:00:00,britishjobsonthe.net
1,69026322,Weekend Engineering Team Leader,essex,non-specified,non-specified,non-specified,Engineering Jobs,39000.000000,2012-05-15 12:00:00,2012-05-29 12:00:00,britishjobsonthe.net
2,67770033,Junior Writer,nottingham,non-specified,non-specified,non-specified,"PR, Advertising & Marketing Jobs",16500.000000,2013-06-17 00:00:00,2013-07-17 00:00:00,britishjobsonthe.net
3,69025763,Net Developer Manchester (Up To pound;****K ...,manchester,non-specified,non-specified,non-specified,IT Jobs,38000.000000,2012-02-21 12:00:00,2012-03-06 12:00:00,britishjobsonthe.net
4,68017718,Chef De Partie pound;****gratutities****AA Ros...,bath,non-specified,non-specified,non-specified,Hospitality & Catering Jobs,16000.000000,2013-03-27 15:00:00,2013-04-10 15:00:00,britishjobsonthe.net


In [26]:
# check the number of df_task3 company names that appear in df_task2
df_task3['Company'].isin(df_task2['Company']).value_counts()

True     4188
False     812
Name: Company, dtype: int64

In [27]:
# where the company name is not in df_task2, save in a different dataframe.
# afterwards, we will get a similarity score.

df_task3_not_in_task2 = df_task3[~df_task3['Company'].isin(df_task2['Company'])]
df_task3_not_in_task2['Company'].value_counts()

                                            486
365                                           6
simpson recruitment                           3
topaz promotions                              2
hafod care association                        2
                                           ... 
keepthinking                                  1
riccardo s restaurant a taste of tuscany      1
staff uk bolton                               1
harris global                                 1
coventry sports foundation                    1
Name: Company, Length: 311, dtype: int64

In [28]:
# HMMMM there are blanks in df_task3!  These will be replaced with 'non-specified'.
# where the company name is blank, replace with 'non-specified'
df_task3['Company'] = np.where(df_task3['Company'] == '', 'non-specified', df_task3['Company'])

df_task3_not_in_task2 = df_task3[~df_task3['Company'].isin(df_task2['Company'])]
df_task3_not_in_task2['Company'].value_counts()

# there are 310 unique company names that appear in task 3 dataset but not in task 2 dataset

365                                         6
simpson recruitment                         3
topaz promotions                            2
m aria recruitment                          2
hafod care association                      2
                                           ..
keepthinking                                1
riccardo s restaurant a taste of tuscany    1
staff uk bolton                             1
harris global                               1
coventry sports foundation                  1
Name: Company, Length: 310, dtype: int64

In [29]:
# Create a DataFrame of unmatched companies
df_task3_not_in_task2 = df_task3[~df_task3['Company'].isin(df_task2['Company'])]

# Find the closest match for each unmatched company name
# Note, that because the data has already been cleaned, the cutt-off is very narrow, 0.95

replace_dict = {}

for comp in df_task3_not_in_task2['Company']:
    closest_match = difflib.get_close_matches(comp, df_task2['Company'], n=1, cutoff=0.95)
    
    if closest_match:
        replace_dict[comp] = closest_match[0]
        print(comp, " --> ",closest_match)

# Now you have a replace_dict that maps unmatched company names to their closest matches
print("\n")
print("Replacement Dictionary length:", len(replace_dict))
print("Replacement Dictionary:", replace_dict)


badenochclark  -->  ['badenoch clark']
pcrecruitment  -->  ['pc recruitment']
sigmar recruitment  -->  ['sigma recruitment']
d p connect  -->  ['dp connect']
harris global  -->  ['harris  global']
express recruitment  -->  ['xpress recruitment']
ashton recruitment  -->  ['aston recruitment']
sigmar recruitment  -->  ['sigma recruitment']
think it recruitment  -->  ['think it reruitment']
madigangill  -->  ['madigan gill']
maw recruitment  -->  ['mw recruitment']
platinum  co  -->  ['platinum co']
logical personel solutions  -->  ['logical personnel solutions']


Replacement Dictionary length: 12
Replacement Dictionary: {'badenochclark': 'badenoch clark', 'pcrecruitment': 'pc recruitment', 'sigmar recruitment': 'sigma recruitment', 'd p connect': 'dp connect', 'harris global': 'harris  global', 'express recruitment': 'xpress recruitment', 'ashton recruitment': 'aston recruitment', 'think it recruitment': 'think it reruitment', 'madigangill': 'madigan gill', 'maw recruitment': 'mw recrui

In [30]:
# Spot check these suggestions and update the location column
df_task3['Company'].replace(replace_dict,inplace=True)

In [31]:
df_task3.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,1,Higher Level Teaching Assistant (HLTA),UK,protocol education,part_time,non-specified,Teaching Jobs,14400.000000,2012-11-27 12:00:00,2012-12-27 12:00:00,www.jobhuntlisting.com
1,2,Software Developers / Software Architects Not...,UK,cordius,full_time,permanent,IT Jobs,37500.000000,2013-06-26 15:00:00,2013-08-25 15:00:00,www.jobhuntlisting.com
2,3,Senior BI Consultant (Cognos),Walton-On-Thames,understanding recruitment,full_time,permanent,IT Jobs,52000.000000,2012-10-18 15:00:00,2013-01-16 15:00:00,www.jobhuntlisting.com
3,4,"Technical Sales Manager, Pigments Fillers to S...",UK,wallace hind selection,full_time,non-specified,Sales Jobs,45000.000000,2012-12-15 15:00:00,2013-02-13 15:00:00,www.jobhuntlisting.com
4,5,Channel Account Manager Software,Eastern England,greenhill group,full_time,non-specified,Sales Jobs,35000.000000,2012-12-29 12:00:00,2013-01-12 12:00:00,www.jobhuntlisting.com


In [32]:
df_task3['Company'].value_counts()

non-specified          486
ukstaffsearch           52
penguin recruitment     34
randstad                28
idex consulting         26
                      ... 
lmr recruitment          1
sewell moorhouse         1
spencer rose             1
howett thorpe            1
impact 360               1
Name: Company, Length: 2237, dtype: int64

In [33]:
# Now we've finished cleaning the company field, let's check for overlap and no overlap
df_task3['Company'].isin(df_task2['Company']).value_counts()

# this is as good as it's going to get.

True     4687
False     313
Name: Company, dtype: int64

### Checking other columns, one last time.

In [34]:
# there are 5 fields which have overlapping data: Salary, Category, Company, Contract Type and Contract time. We can merge on these.
# check if there are null values in each of these 5 columns
df_task3.isnull().sum()

Id                 0
Title              0
Location           0
Company            0
ContractType       0
ContractTime       0
Category        1188
Salary             0
OpenDate           0
CloseDate          0
SourceName         0
dtype: int64

In [35]:
df_task2.isnull().sum()

Id              0
Title           0
Location        0
Company         0
ContractType    0
ContractTime    0
Category        0
Salary          0
OpenDate        0
CloseDate       0
SourceName      0
dtype: int64

In [36]:
#ok! yes there are in 'category'
# Let's change this to unspecified. 
df_task3['Category'] = np.where(df_task3['Category'].isnull(), 'non-specified', df_task3['Category'])

### 3. Finding global key for the data

In this task, we are finding consistent fields that identify a unique job and assigning a concatenation as a temporary global key. This will be used to merge datasets and identify duplicates, then will eventually be replaced with a numerical global key.

In [37]:
# we will create a new column called 'global_id' which will be a concatenation of these 5 fields.
# Once the tables are merged and deduplicated, we will then replace the concatenation with a unique id.

# create a new column called 'global_id' which will be a concatenation of these 5 fields (for now).
df_task3['concat_id'] = df_task3['Salary'].astype(str) + df_task3['Category'] + df_task3['Company'] + df_task3['ContractType'] + df_task3['ContractTime']

# do the same for df_task2
df_task2['concat_id'] = df_task2['Salary'].astype(str) + df_task2['Category'] + df_task2['Company'] + df_task2['ContractType'] + df_task2['ContractTime']


### 4. Merging data and removing duplicates

In this task, after we have created a concatenation global key of 5 key fields, the data is merged. Duplications are identified and removed. Finally, the global key made of the concatenation of 5 key fields is replaced with a numerical global key. 

In [38]:
# merge to create a unified dataset where there are no job duplicates. this means we need to create a unique id for each job
# and then search the unified df for duplicates and remove it.

In [39]:
df_task3.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,concat_id
0,1,Higher Level Teaching Assistant (HLTA),UK,protocol education,part_time,non-specified,Teaching Jobs,14400.000000,2012-11-27 12:00:00,2012-12-27 12:00:00,www.jobhuntlisting.com,14400.0Teaching Jobsprotocol educationpart_tim...
1,2,Software Developers / Software Architects Not...,UK,cordius,full_time,permanent,IT Jobs,37500.000000,2013-06-26 15:00:00,2013-08-25 15:00:00,www.jobhuntlisting.com,37500.0IT Jobscordiusfull_timepermanent
2,3,Senior BI Consultant (Cognos),Walton-On-Thames,understanding recruitment,full_time,permanent,IT Jobs,52000.000000,2012-10-18 15:00:00,2013-01-16 15:00:00,www.jobhuntlisting.com,52000.0IT Jobsunderstanding recruitmentfull_ti...
3,4,"Technical Sales Manager, Pigments Fillers to S...",UK,wallace hind selection,full_time,non-specified,Sales Jobs,45000.000000,2012-12-15 15:00:00,2013-02-13 15:00:00,www.jobhuntlisting.com,45000.0Sales Jobswallace hind selectionfull_ti...
4,5,Channel Account Manager Software,Eastern England,greenhill group,full_time,non-specified,Sales Jobs,35000.000000,2012-12-29 12:00:00,2013-01-12 12:00:00,www.jobhuntlisting.com,35000.0Sales Jobsgreenhill groupfull_timenon-s...


In [40]:
df_task2.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,concat_id
0,67770713,CNC Programmer (Offline),birmingham,non-specified,non-specified,non-specified,Engineering Jobs,32000.000000,2013-06-10 15:00:00,2013-07-10 15:00:00,britishjobsonthe.net,32000.0Engineering Jobsnon-specifiednon-specif...
1,69026322,Weekend Engineering Team Leader,essex,non-specified,non-specified,non-specified,Engineering Jobs,39000.000000,2012-05-15 12:00:00,2012-05-29 12:00:00,britishjobsonthe.net,39000.0Engineering Jobsnon-specifiednon-specif...
2,67770033,Junior Writer,nottingham,non-specified,non-specified,non-specified,"PR, Advertising & Marketing Jobs",16500.000000,2013-06-17 00:00:00,2013-07-17 00:00:00,britishjobsonthe.net,"16500.0PR, Advertising & Marketing Jobsnon-spe..."
3,69025763,Net Developer Manchester (Up To pound;****K ...,manchester,non-specified,non-specified,non-specified,IT Jobs,38000.000000,2012-02-21 12:00:00,2012-03-06 12:00:00,britishjobsonthe.net,38000.0IT Jobsnon-specifiednon-specifiednon-sp...
4,68017718,Chef De Partie pound;****gratutities****AA Ros...,bath,non-specified,non-specified,non-specified,Hospitality & Catering Jobs,16000.000000,2013-03-27 15:00:00,2013-04-10 15:00:00,britishjobsonthe.net,16000.0Hospitality & Catering Jobsnon-specifie...


In [41]:
# Append the two data frames together into a unique new dataframe
df_joined = df_task3.append(df_task2, ignore_index=True)

/var/folders/hn/8z124v9x1x53hqvr0c989hsr0000gn/T/ipykernel_94658/418660398.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_joined = df_task3.append(df_task2, ignore_index=True)


In [42]:
# find duplicates based on the new global_id column
df_joined[df_joined.duplicated(['concat_id'], keep=False)].sort_values(by=['concat_id'])

# drop duplicates based on the new global_id column. keep the first instance.
df_joined.drop_duplicates(subset=['concat_id'], keep='first', inplace=True)


In [43]:
# check again for duplicates based on the new global_id column
df_joined[df_joined.duplicated(['concat_id'], keep=False)].sort_values(by=['concat_id'])

# Nothing! Hurrah!

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,concat_id


In [44]:
# Now we are going to create a global ID and remove the concat_id column
df_joined['global_id'] = range(1000, 1000 + len(df_joined))

# drop the concat_id column
df_joined.drop(['concat_id'], axis=1, inplace=True)
df_joined.head()


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,global_id
0,1,Higher Level Teaching Assistant (HLTA),UK,protocol education,part_time,non-specified,Teaching Jobs,14400.000000,2012-11-27 12:00:00,2012-12-27 12:00:00,www.jobhuntlisting.com,1000
1,2,Software Developers / Software Architects Not...,UK,cordius,full_time,permanent,IT Jobs,37500.000000,2013-06-26 15:00:00,2013-08-25 15:00:00,www.jobhuntlisting.com,1001
2,3,Senior BI Consultant (Cognos),Walton-On-Thames,understanding recruitment,full_time,permanent,IT Jobs,52000.000000,2012-10-18 15:00:00,2013-01-16 15:00:00,www.jobhuntlisting.com,1002
3,4,"Technical Sales Manager, Pigments Fillers to S...",UK,wallace hind selection,full_time,non-specified,Sales Jobs,45000.000000,2012-12-15 15:00:00,2013-02-13 15:00:00,www.jobhuntlisting.com,1003
4,5,Channel Account Manager Software,Eastern England,greenhill group,full_time,non-specified,Sales Jobs,35000.000000,2012-12-29 12:00:00,2013-01-12 12:00:00,www.jobhuntlisting.com,1004


In [45]:
df_joined.tail()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,global_id
55748,68704692,Graduate SoftwareWeb Developer,london,unknown auto added by vacany import,non-specified,non-specified,IT Jobs,25000.000000,2012-01-12 12:00:00,2012-03-12 12:00:00,britishjobsonthe.net,44933
55749,72653751,Barclays Future Leaders Development Programmes,hackney,barclays,non-specified,non-specified,IT Jobs,36000.000000,2013-07-21 15:00:00,2013-09-19 15:00:00,britishjobsonthe.net,44934
55750,72420369,Graduate Operations Technical Analyst,hungerford,unknown auto added by vacany import,non-specified,non-specified,IT Jobs,23000.000000,2013-09-13 15:00:00,2013-11-12 15:00:00,britishjobsonthe.net,44935
55751,72653749,GCHQ Graduate Technologists various roles,cheltenham,gchq,non-specified,non-specified,Engineering Jobs,25400.000000,2012-04-12 00:00:00,2012-06-11 00:00:00,britishjobsonthe.net,44936
55752,66372417,Graduate Engineer (Water industry),south lanarkshire,carrington west,full_time,contract,Engineering Jobs,20000.000000,2013-01-19 00:00:00,2013-04-19 00:00:00,britishjobsonthe.net,44937


In [46]:
df_joined.describe()

,Id,Salary,global_id
count,43938.000000,43938.000000,43938.000000
mean,61826368.207952,34451.992671,22968.500000
std,21633340.772600,16033.715201,12683.952401
min,1.000000,5000.000000,1000.000000
25%,67959602.250000,23000.000000,11984.250000
50%,69078965.500000,31200.000000,22968.500000
75%,71172090.500000,42500.000000,33952.750000
max,72705203.000000,108725.000000,44937.000000


### 5. Saving the integrated and reshaped data
The last part of the integration process is to export our output data to csv format, named as:
- '\<student\_id\>_dataset_integrated.csv'

In [47]:
# code to save output data
df_joined.to_csv('s3932835_dataset_integrated.csv', index=False)

## Conclusion of Task 3



In conclusion, Task 3 posed its own set of challenges in the final step of this data integration journey. The primary objective was to merge job posting data from a separate job posting source with the cleaned dataset derived from the outputs of Task 1 and 2. The aim was to create a unified, duplicate-free dataset based on a global key.

However, numerous issues were encountered that had to be addressed before merging. One significant challenge was dealing with differences in field names and values between the two datasets. To address this, I implemented normalisation techniques, ensuring that similar fields were aligned correctly, especially in cases like category fields.

Another issue was the presence of NULL values in the provided dataset. While this required minor cleaning, it was crucial to maintain data integrity.

The comparison of company names across the two datasets revealed slight variations that needed reconciliation. Cross-checks were conduction between Task 2 and Task 3 datasets and similarity score analysis was used to identify and match similar company names, ensuring consistency in our final dataset.

Furthermore, after merging the data, duplicates were checked for to eliminate unnecessary data, ensuring the integrity and quality of the joined dataset.

To overcome these challenges, we used employed univariate analysis, renamed fields, converted numerical categorical values to text, and created a global data key. These steps were essential in achieving the goal of creating a clean, unified dataset that seamlessly integrated data from multiple sources.

In summary, Task 3 presented its unique data integration challenges, but through a systematic and data-driven approach, the datasets were successfully merged, resulting in a consolidated and high-quality dataset ready for further analysis and insights. This task highlights the importance of careful data integration to ensure the reliability of the final dataset in any data-related project.